In [1]:
import pandas as pd
import plotly.express as px
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import string
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv('UpdatedResumeDataSet.csv')
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [5]:
print(df.shape)
df.drop_duplicates(subset="Resume", keep='first', inplace=True)

(962, 2)


In [6]:


fig = px.histogram(df, x="Category", title="Distribution of Job Categories", category_orders={"Category": df["Category"].value_counts().index})

fig

In [7]:
series = df['Category'].value_counts()

df_result = pd.DataFrame(series)

df_result = df_result.reset_index()  

df_result.columns = ['Category', 'Total']

print(df_result)

                     Category  Total
0              Java Developer     13
1                    Database     11
2                Data Science     10
3                    Advocate     10
4                          HR     10
5            DotNet Developer      7
6                      Hadoop      7
7             DevOps Engineer      7
8          Automation Testing      7
9                     Testing      7
10             Civil Engineer      6
11           Business Analyst      6
12              SAP Developer      6
13         Health and fitness      6
14           Python Developer      6
15                       Arts      6
16     Electrical Engineering      5
17                      Sales      5
18  Network Security Engineer      5
19        Mechanical Engineer      5
20              ETL Developer      5
21                 Blockchain      5
22         Operations Manager      4
23              Web Designing      4
24                        PMO      3


In [8]:
fig = px.pie(df_result,
                 values='Total',
                 names='Category')

fig

In [13]:
import spacy
import re
from spacy.lang.en.stop_words import STOP_WORDS

# Load spaCy model
nlp = spacy.load('en_core_web_sm')

# Define additional stopwords that might be relevant for resumes
additional_stopwords = {'resume', 'cv', 'http', 'https', 'www'}

# Update spaCy's default stopwords list with the additional stopwords
for word in additional_stopwords:
    nlp.vocab[word].is_stop = True

def cleanResume_spacy(resumeText, normalize_case=True):
    # Remove email addresses
    resumeText = re.sub(r'\S*@\S*\s?', '', resumeText)
    # Remove URLs
    resumeText = re.sub(r'http\S+', '', resumeText)
    # Additional regex-based clean-up could be placed here if necessary

    doc = nlp(resumeText)
    # Process tokens
    tokens = []
    for token in doc:
        if not token.is_stop and not token.is_punct and token.is_alpha:
            # Check if token is a proper noun or not
            if normalize_case and token.pos_ != 'PROPN':
                # Normalize case for non-proper nouns
                tokens.append(token.lemma_.lower())
            else:
                # Keep the original token text for proper nouns or if normalization is not desired
                tokens.append(token.lemma_ if token.lemma_ != '-PRON-' else token.text)
    return " ".join(tokens)

# Example application of the function with case normalization
df['cleaned_resume'] = df.Resume.apply(lambda x: cleanResume_spacy(x, normalize_case=True))


In [14]:
df.head()

,Category,Resume,cleaned_resume
0,Data Science,Skills * Programming Languages: Python (pandas...,skill Programming language Python pandas numpy...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,Education Details UIT RGPV Data Scientist Data...
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",area Interest Deep Learning Control System Des...
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,skill r Python SAP HANA Tableau SAP hana SQL S...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",Education Details MCA YMCAUST Faridabad Haryan...


In [19]:
def get_document_vector(text):
    doc = nlp(text)
    return doc.vector

# Assuming you decide to use document vectors directly as features
df['document_vector'] = df['cleaned_resume'].apply(get_document_vector)

# Use document vectors as features for classification
X = df['document_vector']
y = df['Category']

# Split data into train and test sets, then train your classifier


In [16]:
oneSetOfStopWords = set(stopwords.words('english')+['``',"''"])
totalWords =[]
Sentences = df['Resume'].values
cleanedSentences = ""
for records in Sentences:
    cleanedText = cleanResume_spacy(records)
    cleanedSentences += cleanedText
    requiredWords = nltk.word_tokenize(cleanedText)
    for word in requiredWords:
        if word not in oneSetOfStopWords and word not in string.punctuation:
            totalWords.append(word)
    
wordfreqdist = nltk.FreqDist(totalWords)
mostcommon = wordfreqdist.most_common(50)
print(mostcommon)

[('Exprience', 592), ('company', 544), ('month', 526), ('Details', 508), ('description', 484), ('project', 310), ('work', 291), ('Project', 279), ('year', 251), ('team', 236), ('datum', 222), ('Maharashtra', 217), ('SQL', 201), ('Â', 196), ('report', 189), ('January', 189), ('database', 189), ('system', 179), ('Skill', 175), ('requirement', 169), ('Management', 158), ('Pune', 157), ('application', 155), ('test', 155), ('process', 154), ('management', 149), ('user', 147), ('client', 145), ('provide', 144), ('responsibility', 143), ('Education', 140), ('customer', 140), ('Data', 133), ('create', 131), ('Engineering', 131), ('Developer', 131), ('business', 127), ('Database', 125), ('monthscompany', 125), ('like', 124), ('issue', 123), ('University', 123), ('Server', 123), ('Java', 122), ('activity', 122), ('Pvt', 121), ('develop', 120), ('Ltd', 117), ('India', 116), ('design', 115)]


In [21]:
def extract_entities(text):
    doc = nlp(text)
    skills = []
    

    for ent in doc.ents:
        if ent.label_ == "SKILL":
            skills.append(ent.text)
        

    return {"skills": list(set(skills))}

# Example usage:
df['entities'] = df['cleaned_resume'].apply(extract_entities)

# Use the extracted entities as additional features for classification


In [22]:
from sklearn.preprocessing import LabelEncoder

var_mod = ['Category']
le = LabelEncoder()
for i in var_mod:
    df[i] = le.fit_transform(df[i])

In [23]:
df.Category.value_counts()

Category
15    13
7     11
6     10
0     10
12    10
9      7
13     7
8      7
2      7
23     7
5      6
4      6
21     6
14     6
20     6
1      6
11     5
22     5
17     5
16     5
10     5
3      5
18     4
24     4
19     3
Name: count, dtype: int64

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import spacy

# Load your spaCy model
nlp = spacy.load('en_core_web_sm')

# Assuming df is your DataFrame and it already has 'cleaned_resume' and 'Category' columns

# Preprocess Document Vectors
df['document_vector'] = df['cleaned_resume'].apply(lambda x: nlp(x).vector)

# Preprocess Entities
# Assuming entities are stored in a column 'entities' and are lists of skills
df['skills'] = df['entities'].apply(lambda x: ', '.join(x['skills']) if x and 'skills' in x else '')

# Convert document vectors into a DataFrame
vector_df = pd.DataFrame(df['document_vector'].to_list())

# Prepare feature matrix X by concatenating vector_df with skills after vectorization/encoding if necessary
# Here, we'll proceed without further encoding of 'skills' for simplicity
X = pd.concat([vector_df, df['skills']], axis=1)

# Prepare target variable y
y = df['Category']

# Drop rows with NaN values in X and y to ensure consistency
X = X.dropna()
y = y[X.index]  # Ensure y has corresponding indices after dropping NaN values in X

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Since sklearn cannot directly handle text features like 'skills' without encoding, 
# we will drop 'skills' for simplicity in this example. In a real scenario, consider applying feature extraction techniques like TF-IDF on 'skills'.
X_train = X_train.drop('skills', axis=1)
X_test = X_test.drop('skills', axis=1)

# Train a RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of RandomForest Classifier on test set: {accuracy:.2f}")


Accuracy of RandomForest Classifier on test set: 0.62


In [33]:
from sklearn.model_selection import GridSearchCV

# Assuming X_train and y_train are prepared without 'skills' feature for simplicity

# Set up the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the classifier
clf = RandomForestClassifier(random_state=42)

# Set up the grid search
grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best parameters and best score
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {grid_search.best_score_:.2f}")

# Evaluate on the test set
best_clf = grid_search.best_estimator_
y_pred = best_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the best RandomForest Classifier on test set: {accuracy:.2f}")


Fitting 5 folds for each of 108 candidates, totalling 540 fits


c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning:

The least populated class in y has only 3 members, which is less than n_splits=5.



Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Best cross-validation score: 0.50
Accuracy of the best RandomForest Classifier on test set: 0.62


In [34]:
import numpy as np

# Assuming 'document_vector' is a column where each row contains an array-like object of features
# Convert list of vectors into a matrix where each row is a document vector
X = np.vstack(df['document_vector'])

# Your labels
y = df['Category'].values


In [35]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Models to evaluate
models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, random_state=42),
    "SVM": make_pipeline(StandardScaler(), SVC(gamma='auto'))
}

# Cross-validation settings
cv_folds = 5

# Evaluate each model using cross-validation
for name, model in models.items():
    cv_scores = cross_val_score(model, X, y, cv=cv_folds, scoring='accuracy')
    print(f"{name} CV accuracy: {np.mean(cv_scores):.2f} +/- {np.std(cv_scores):.2f}")


c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning:

The least populated class in y has only 3 members, which is less than n_splits=5.



RandomForest CV accuracy: 0.22 +/- 0.06


c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning:

The least populated class in y has only 3 members, which is less than n_splits=5.



GradientBoosting CV accuracy: 0.14 +/- 0.06
SVM CV accuracy: 0.16 +/- 0.03


c:\Python311\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning:

The least populated class in y has only 3 members, which is less than n_splits=5.

